# Masker Module

In [1]:
#| default_exp masker

In [2]:
#| export 
import CodeCheckList.utils as utils
from CodeCheckList.tokenizer import CodeTokenizer
from tree_sitter import Parser
import random

/home/svelascodimate/miniconda3/envs/code-check-list/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export 
class Masker():
    """Masker module"""
    def __init__(self, code_tokenizer: CodeTokenizer):
        self.code_tokenizer = code_tokenizer

    def __call__(
    self,                            #self
    code: str,                       #source code snippet to mask
    encoding: list,                  #list of encodings
    target_node_type_id: int,        #target node type id to search, -1: random masking
    masking_rate: float,             #masking rate to apply [0-1]
    random_masking: float = False,   #random_masking, false by default
    ):  
        node_mask_id = self.code_tokenizer.tokenizer.mask_token_id
        if random_masking:
            positions = random.sample(range(len(encoding['input_ids'])), int(masking_rate*len(encoding['input_ids'])))
            for idx in positions:
                encoding['input_ids'][idx]=node_mask_id
            return encoding
        tree = self.code_tokenizer.parser.parse(bytes(code, "utf8"))
        filtered_nodes = []
        utils.find_nodes(tree.root_node, self.code_tokenizer.node_types[target_node_type_id], filtered_nodes)
        filtered_node_offsets = [(utils.convert_to_offset(node.start_point, code.split("\n")), 
            utils.convert_to_offset(node.end_point, code.split("\n"))) for node in filtered_nodes]
        filtered_node_offsets = utils.get_elements_by_percentage(filtered_node_offsets, masking_rate)
        for index, offset in enumerate(encoding['offset_mapping']):
            for filtered_node_offset in filtered_node_offsets:
                if offset[0] >= filtered_node_offset[0] and offset[1]<= filtered_node_offset[1]:
                    encoding['input_ids'][index] = node_mask_id
        
        return encoding